1. Project Title & Team Information

Title:  Accident Risk Analysis: Urban vs. Rural Populations

Team Members: Dave Woodford, Tim Mai, Matt Dolin, and Phillip Roman

#TO DO: Team Members Table (Name, Role, Skills, Contribution)

2. Project Definition & Motivation

Objective: "Does accident frequency increase in high-population counties? Do factors differ between rural and metro areas?"

Stakeholders: Dept of Transporation, Railroad Administration, Railroad companies, local municipaliteis, and the public

Potential Application: A risk-assessment tool or "safe travel" dashboard


# 3. Data Sources & Selection

To investigate the relationship between population density and rail safety, we selected two primary datasets that allow for a granular geospatial analysis at the county level.

### **Source A: Rail Equipment Accident/Incident Data (Form 54)**
* **Publisher:** Federal Railroad Administration (FRA) / Department of Transportation (DOT)
* **Period:** 2012 - Present (Filtered for schema consistency)
* **URL:** [DOT Open Data Portal](https://data.transportation.gov/Railroads/Rail-Equipment-Accident-Incident-Data-Form-54-/85tf-25kj/about_data)
* **Justification & Selection Criteria:**
    * **Authoritative Record:** This dataset is the federal standard for reportable rail accidents.
    * **Reporting Threshold:** We acknowledge that this dataset excludes minor incidents. Inclusion is based on a monetary reporting threshold (adjusted annually for inflation, approx. $12,000+ in recent years), ensuring our analysis focuses on events with significant economic impact or infrastructure damage.
    * **Temporal Cut-off:** We restricted our analysis to data starting from **2012**. While the FRA updated their reporting schema in June 2011, we excluded the entire transition year to ensure maximum data integrity and consistent definitions for key variables like `TERRITORY`.

### **Source B: Rural-Urban Continuum Codes (2023)**
* **Publisher:** USDA Economic Research Service (ERS)
* **Period:** 2023 Edition
* **URL:** [USDA ERS Data Products](https://www.ers.usda.gov/data-products/rural-urban-continuum-codes)
* **Justification (USDA vs. US Census):**
    * **Superior Granularity:** Standard US Census data relies on simple population counts. The USDA RUCC system classifies counties into a 9-point scale based on distinct *environmental* factors.
    * **The "Adjacency" Factor:** Crucially, this dataset distinguishes rural counties by their **adjacency to metropolitan areas**. This is vital for our analysis, as we hypothesize that rural counties adjacent to major transit hubs face different rail traffic risks than isolated rural counties—a nuance that standard census counts would miss.

In [13]:
# Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="whitegrid")

# Load data
rail_accident_path = "../data/raw/Rail_Equipment_Accident_Incident_Data_(Form_54)_20260114.csv.zip"
population_path = "../data/raw/Ruralurbancontinuumcodes2023.csv"

df_accident_all = pd.read_csv(
    rail_accident_path,
    low_memory=False
)

df_population = pd.read_csv(
    population_path,
    encoding="latin1"
)

# Checks
print(f"Rail Data Loaded. Shape: {df_accident_all.shape}")
print(f"Population Data Loaded. Shape: {df_population.shape}")
print()
display(df_accident_all.head(3))
print()
display(df_population.head(3))

Rail Data Loaded. Shape: (223529, 155)
Population Data Loaded. Shape: (9703, 5)



,Reporting Railroad Code,Reporting Railroad Name,Year,Accident Number,PDF Link,Accident Year,Accident Month,Other Railroad Code,Other Railroad Name,Other Accident Number,...,Reporting Parent Railroad Name,Reporting Railroad Holding Company,Reporting Railroad Individual Class,Reporting Railroad Passenger,Reporting Railroad Commuter,Reporting Railroad Switching Terminal,Reporting Railroad Tourist,Reporting Railroad Freight,Reporting Railroad Short Line,Location
0,SOO,SOO Line Railroad Company,1981,CA28,https://safetydata.fra.dot.gov/Officeofsafety/...,81,8,NaN,NaN,NaN,...,CANADIAN PACIFIC KANSAS CITY,Not Assigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,NaN
1,MP,Missouri Pacific Railroad Company,1981,81230,https://safetydata.fra.dot.gov/Officeofsafety/...,81,9,NaN,NaN,NaN,...,Union Pacific Railroad Company,Union Pacific Railroad Company,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,NaN
2,SP,Southern Pacific Transportation Company,1980,W1200,https://safetydata.fra.dot.gov/Officeofsafety/...,80,1,NaN,NaN,NaN,...,Union Pacific Railroad Company,Union Pacific Railroad Company,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,NaN


,FIPS,State,County_Name,Attribute,Value
0,1001,AL,Autauga County,Population_2020,58805
1,1001,AL,Autauga County,RUCC_2023,2
2,1001,AL,Autauga County,Description,"Metro - Counties in metro areas of 250,000 to ..."


# 4. Methodology: Data Ingestion & Cleaning

Our analysis requires merging two disparate datasets that lack a common unique identifier in their raw state. The Rail Accident data uses separate columns for "State" and "County," while the Census data uses a unified 5-digit FIPS code.

**Data Cleaning Pipeline:**
1.  **Temporal Filtering:** We filtered the dataset to include only years **> 2011**, avoiding the schema inconsistencies of the pre-2011 reporting standards.
2.  **Null Removal:** Rows with missing "County Code" identifiers were removed to ensure accurate geospatial mapping, as these records cannot be linked to census data.
3.  **FIPS Construction:** We synthesized a 5-digit Federal Information Processing Standard (FIPS) code for the rail data by padding the State Code (2 digits) and County Code (3 digits).
4.  **Pivoting:** The Census data was pivoted from a "Long" format (multiple rows per county) to a "Wide" format (one row per county) to expose `Population_2020` and `Description` as distinct columns.

In [22]:
df_accident = df_accident_all.copy()

# Filter for accidents after 2011
df_accident = df_accident[df_accident["Year"] > 2011]

# Clean railroad accident data
df_accident = df_accident.dropna(subset=["County Code"])
df_accident["State Code FIPS"] = df_accident["State Code"].astype(int).astype(str).str.zfill(2)
df_accident["County Code FIPS"] = df_accident["County Code"].astype(int).astype(str).str.zfill(3)
df_accident["FIPS"] = (df_accident["State Code FIPS"] + df_accident["County Code FIPS"]).astype(int)

# Clean population data
df_population_wide = df_population.pivot(index="FIPS", columns="Attribute", values="Value").reset_index()
df_population_wide["FIPS"] = df_population_wide["FIPS"].astype(int)
df_population_wide.columns.name = None

# Verify population is numeric - remove if not
if "Population_2020" in df_population_wide.columns:
    df_population_wide["Population_2020"] = (
        pd.to_numeric(df_population_wide["Population_2020"].astype(str).str.replace(",", ""), errors="coerce")
    )

# Checks
print("Railroad Data: FIPS Check")
cols_to_verify = ["State Code", "County Code", "State Code FIPS", "County Code FIPS", "FIPS"]
display(df_accident[cols_to_verify].head())

print("\nPopulation Data: Pivot Check - wide")
display(df_population_wide.head())


Railroad Data: FIPS Check


,State Code,County Code,State Code FIPS,County Code FIPS,FIPS
50,53,53.0,53,053,53053
54,21,15.0,21,015,21015
56,47,37.0,47,037,47037
58,45,47.0,45,047,45047
1159,12,99.0,12,099,12099



Population Data: Pivot Check - wide


,FIPS,Description,Population_2020,RUCC_2023
0,1001,"Metro - Counties in metro areas of 250,000 to ...",58805,2
1,1003,Metro - Counties in metro areas of fewer than ...,231767,3
2,1005,"Nonmetro - Urban population of 5,000 to 20,000...",25223,6
3,1007,Metro - Counties in metro areas of 1 million p...,22293,1
4,1009,Metro - Counties in metro areas of 1 million p...,59134,1


### **5. The Merge Strategy**

We merged the Rail Accident data with the Census data using a **Left Join** on the generated `FIPS` key.

* **Linkage:** Accidents were matched to their corresponding county population and rural-urban classification.
* **Exclusion of Unmatched Records:** Following the merge, we removed any accident records that failed to match with a valid census entry (i.e., where `Population_2020` was null). This step is necessary because accidents occurring in unidentified counties cannot be normalized for follow-on per-capita risk analysis.

In [21]:
# Merge
df_merged = pd.merge(
    df_accident,
    df_population_wide,
    on="FIPS",
    how="left"
)

# Drop rows/accidents where population is blank
df_merged = df_merged.dropna(subset=["Population_2020"])

# Checks
print(f"Merged Shape: {df_merged.shape}")
print("New columns available:", list(df_population_wide.columns))
cols_to_show = ["FIPS", "State Name", "Description", "Population_2020"]
display(df_merged[cols_to_show].head())

Merged Shape: (34562, 161)
New columns available: ['FIPS', 'Description', 'Population_2020', 'RUCC_2023']


,FIPS,State Name,Description,Population_2020
0,53053,WASHINGTON,Metro - Counties in metro areas of 1 million p...,921130.0
1,21015,KENTUCKY,Metro - Counties in metro areas of 1 million p...,135968.0
2,47037,TENNESSEE,Metro - Counties in metro areas of 1 million p...,715884.0
3,45047,SOUTH CAROLINA,"Nonmetro - Urban population of 20,000 or more,...",69351.0
4,12099,FLORIDA,Metro - Counties in metro areas of 1 million p...,1492191.0


6. Initial Exploratory Analysis (EDA)

The Question: "Is there a correlation between population size and raw accident count?"

The Visualization: Your scatter plot (plt.scatter(x,y)).

The Caption/Interpretation: "The plot shows a positive correlation (0.58), but outliers exist. This suggests that while population is a factor, other variables (track density, traffic volume) likely play a role."

7. Limitations and Future Improvements

Current Limitations: The analysis currently looks at raw counts, not normalized by "train miles" or "track miles" (traffic density).

Missing Variables: Weather, Time of Day, Seasonality, etc. 

### **7.1 Data Nuances & Methodology Notes**
* **Reporting Threshold Bias:** Our accident counts are inherently filtered by the FRA's monetary reporting threshold (CFR Part 225). Accidents resulting in damages below this inflation-adjusted threshold (e.g., minor derailments in yards) are not present in the dataset. Our conclusions therefore apply specifically to *significant* rail accidents rather than all operational irregularities.
* **Schema Consistency (2011):** We explicitly excluded data prior to June 2011 due to changes in the FRA's reporting requirements. Merging pre-2011 data would introduce inconsistencies in variable definitions (specifically regarding track territory and injury classification), potentially skewing the risk profile.

8. Dissemination & Next Steps

Target Audience: 
    US Population (safety consciousness)
    Gov Agencies (maintenance and policy prioritization)
    Railroad companies

Next Steps:
    Analyzing specific "Cause Codes" (Human error vs. Equipment failure) across those bins.